# Iterables vs. Iterators vs. Generators 

## Let's explore the differences between...
* a container
* an iterable
* an iterator
* a generator
* a generator expression
* a {list, set, dict} comprehension


![alt-text](relationships.png)

## Containers 
* data structures which hold elements
* support membership tests
* live in memory
* typically hold all their values in memory
* e.g., string, list, tuple, set, dict
* an object is a container when it can be asked whether it _contains_ a certain element

In [1]:
1 in [1, 2, 3], 0 in [1, 2, 3]

(True, False)

In [2]:
4 in {4, 5, 6}, 1 in {4, 5, 6}

(True, False)

In [3]:
44 in ('Obama', 'Barack', 44, 2008, 'left')

True

In [4]:
# for dicts, membership checks the keys, not the values
44 in { 43: 'Bush', 44: 'Obama'}, 'Bush' in { 43: 'Bush', 44: 'Obama' }

(True, False)

In [5]:
'J' in 'Steve Jobs', 'Job' in 'Steve Jobs', 'Jobs' not in 'Carlos Jobim'

(True, True, True)

## Iterables
* any object, not necessarily a data structure, that can return an iterator (with the purpose of returning all of its elements)
* the `__iter__()` function returns an iterator
    * ...therefore, any object which has the `__iter__()` method is an iterable
* most containers are also iterable
* many more things are iterable as well (e.g., open files, open sockets, etc.)

In [6]:
mylist = [1, 2, 3]
iter1 = iter(mylist)
print(type(iter1))
iter2 = mylist.__iter__() # iter() maps to __iter__()
print(type(iter2))
next(iter1)

&lt;class &#39;list_iterator&#39;&gt;
&lt;class &#39;list_iterator&#39;&gt;


1

In [7]:
iter1.__next__() # next() maps to __next__()
#next(iter1)

2

In [8]:
next(iter1)

3

In [9]:
next(iter1)

StopIteration: 

In [10]:
next(iter2)

1

In [11]:
type(mylist), type(iter1), type(iter2)

(list, list_iterator, list_iterator)

In [12]:
# a list is iterable, but it is not its own iterator
next(mylist)

TypeError: &#39;list&#39; object is not an iterator

In [13]:
iter(mylist) is mylist

False

In [14]:
mylistiter = iter(mylist)
print('%x %x' % (id(mylist), id(mylistiter)))

7f9c97632040 7f9c97664910


In [15]:
iter(mylistiter) is mylistiter

True

## When we write...
`mylist = [1, 2, 3]
for x in mylist:
    ...`
## ...this is what happens

![alt-text](iterable.png "iterable")

## We can see this by disassembling the Python code...

In [16]:
import dis
mylist = [1, 2, 3]
total = 0
dis.dis('for item in mylist: total += item')

  1           0 LOAD_NAME                0 (mylist)
              2 GET_ITER
        &gt;&gt;    4 FOR_ITER                12 (to 18)
              6 STORE_NAME               1 (item)
              8 LOAD_NAME                2 (total)
             10 LOAD_NAME                1 (item)
             12 INPLACE_ADD
             14 STORE_NAME               2 (total)
             16 JUMP_ABSOLUTE            4
        &gt;&gt;   18 LOAD_CONST               0 (None)
             20 RETURN_VALUE


## So what is an iterator?
* a stateful object that produces the next value when you call __`next()`__ on it
* any object that has a __\_\_`next`\_\_()__ method is therefore an iterator
* how it produces a value is irrelevant
* in other words, an iterator is a value factory
 * each time you ask it for "the next" value, it knows how to compute it because it holds internal state

In [17]:
# let's see how an iterator works...
mylist = [13, 46, -3, 'Go!']
myiter = iter(mylist) # get the list iterator

try:
    while True:
        val = next(myiter)
        print(val, end=' ')
except StopIteration:
    print('Stop!')

13 46 -3 Go! Stop!


## Let's build our own iterator!

In [18]:
class Fibonacci(object):
    def __init__(self):
        self.prev = 0
        self.curr = 1

    def __iter__(self):
        return self

    def __next__(self):
        """ 
        each call to next() does two important things:

        1. modify its state for the subsequent next() call
        2. produces a result for the current call
        """
        value = self.curr
        self.curr += self.prev
        self.prev = value
        if value > 1000:
            raise StopIteration
        return value

# Note that this class is both an iterable due to __iter__()
# method and its own iterator, due to __next__() method!

f = Fibonacci()
print(next(f), next(f), 'before the for loop')

for num in f:
    print(num, end=' ')

1 1 before the for loop
2 3 5 8 13 21 34 55 89 144 233 377 610 987 

## Lab: Iterators

Write your own iterator class which takes an iterable and each time it's invoked, it returns a *random* element. The iterator should stop (i.e., __`raise`__ the __`StopIteration`__ exception) when it has returned all elements of the iterable.

Example: __`MyRandomIterator([1, 2, 3])`__ might return

`
2
3
1
...then raise StopIteration`

## Generators
* a generator allows you to write iterators much like the Fibonacci iterator above but in an elegant, succinct syntax that avoids writing classes with __\_\_`iter`\_\_`()`__ and __\_\_`next`\_\_`()`__ methods
* every generator is an iterator (but not vice versa!) 
* a generator is a factory that lazily produces values
* two types: generator _functions_ and generator _expressions_

## The `yield` statement
* before we jump into generators, let's take an in-depth look at what makes them possible...
* when a normal Python function is invoked, execution starts at the first line and continues until a `return` statement is encountered or an exception is thrown (remember that "falling off the end of the function" is the same as if we had written __`return None`__)
    * once a function returns, that's it–any work done by the function and stored in local variables is lost
    * the next call to the function starts everything anew
* there are times when we'd like to have a "function" which yields a series of values, i.e., it would have to save its state to that the next time it's invoked, it picks up where it left off
    * we use the term "yield" here because in fact we are *not returning* to the caller i.e., we are not returning control of execution to the point where the function was called
    * instead of __`return`__-ing, we are __`yield`__-ing, which implies that the transfer of control is temporary and voluntary–our function expects to regain control in the future
* functions that use __`yield`__ instead of __`return`__ are generator functions (or *coroutines* in other languages)
* think of __`yield`__ as __`return`__ + "some magic" for generator functions

### So what's the magic?
* when __`yield`__ is called the state of the generator function is recorded
    * the value of all variables are saved
    * the next line of code to be executed is also saved
    * i.e., the function simply resumes where it left off

In [1]:
def simple_generator():
    yield 1
    yield 'boo!'
    yield 3
    
for value in simple_generator():
    print(value)

1
boo!
3


## Why do we need generator functions?
* initially they gave programmers an easy way to write code that produced a series of values
    * without generator functions, writing something like a random number generator required a class or module that both generated values and kept track of state between calls
    * with generator functions, doing the above is greatly simplified
* suppose we want a function which, given a list of numbers, returns a list of those numbers which are prime
    * straighforward...
    
             def get_primes(nums):
                 return ([num for num in nums if is_prime(num)])

* now suppose we want to use the above function for very large lists of numbers...so large, in fact, that they won't fit in memory
    * so now we want the function to take a starting value, and return all the primes that are greater than that value
    * since functions only return once, they only have one "chance" to return a value (or list of values)
    * what if our function could return the *next* value, rather than a list?
        * we wouldn't need to create a list at all!

## What is a generator function?
* defined like a normal function, but whenever it needs to generate a value, it does so with the __`yield`__ keyword rather than __`return`__
    * if the body of a def contains __`yield`__, the function automatically becomes a generator function (even if it also has a __`return`__ statement)
    * ...there's nothing else we need to do to create one
* generator functions create *generator iterators* (or simply, a *generator*)
    * a generator is a special type of iterator (meaning it has a __\_\_`next`\_\_`()`__ function)
    * to get the next value from a generator, we use the same built-in function as for iterators: __`next()`__
* let's return to the more basic notion of a generator function...

### Now we can rewrite our `get_primes()` function as a generator...

    def get_primes(num):
        while True:
            if is_prime(num):
                yield num
            num += 1
            
* note that if a generator function calls return (or simply hits the end of the function), then a __`StopIteration`__ exception is raised, signaling the generator is exhausted (just as an iterator does)

In [2]:
def fibonacci():
    '''
    defined as a normal function, but...
    ...no return keyword
    
    The yield keyword returns a value, but the function retains its state
    '''
    prev, curr = 0, 1
    while True:
        yield curr
        prev, curr = curr, prev + curr
        
f = fibonacci()
print(next(f), next(f), 'before the for loop', sep='\n')

import random

for num in range(0, random.randint(10, 100)):
    val = next(f)
    print(val, end=' ')

1
1
before the for loop
2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 165580141 267914296 433494437 701408733 1134903170 1836311903 2971215073 4807526976 7778742049 12586269025 20365011074 32951280099 53316291173 86267571272 

## PEP-342: sending values into generators
* PEP-342 added support for passing values *into* generators using the `send()` function
* let's go back to the prime number example but instead of simply printing every prime number greater than some number, we'll find the smallest prime number greater than successive powers of a number (i.e. for 10, we want the smallest prime greater than 10, then 100, then 1000, etc.)

        def get_primes(num):
            while True:
                if is_prime(num):
                    num = yield num 
                num += 1
                
* the `yield` line now says __"yield num, and when a value is sent to me, set num to that value"__
* and we can print the next prime greater than 10, 100, 1000, as follows:

        def print_successive_primes(iterations, base=10):
            prime_generator = get_primes(base)
            prime_generator.send(None)
            for power in range(iterations):
                print(prime_generator.send(base ** power))
                
* printing __`generator.send()`__ is possible because __`send`__ both sends a value to the generator and returns the value yielded by the generator
* note that the first time we send a value into a generator, it must be __`None`__

## Now let's look at a generator _expression_
* generator equivalent of a list comprehension

In [3]:
squares = [num * num for num in range(1, 11)] # list comprehension
squares

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [4]:
squares = {num * num for num in range(1, 11)} # set comprehension
squares

{1, 4, 9, 16, 25, 36, 49, 64, 81, 100}

In [5]:
squares = {num: num * num for num in range(1, 11)} # dict comprehension
squares

{1: 1, 2: 4, 3: 9, 4: 16, 5: 25, 6: 36, 7: 49, 8: 64, 9: 81, 10: 100}

In [6]:
squares = (num * num for num in range(1, 11)) # generator expression (NOT a 'tuple comprehension')
squares

<generator object <genexpr> at 0x7f7f83e713c0>

In [7]:
next(squares), next(squares)

(1, 4)

In [8]:
list(squares) # for thing in squares: print(thing)

[9, 16, 25, 36, 49, 64, 81, 100]

## Lab: Generators
* modify your random iterator to be a generator function

## __`itertools`__
* module of functions for efficient looping
* all of its functions return iterators
* some produce finite sequences
* others produce infinite sequences

In [1]:
from itertools import zip_longest
list1 = ['a', 'b', 'c', 'd']
list2 = ['apple', 'banana', 'cherry']

for item1, item2 in zip_longest(list1, list2, fillvalue='***'):
    print(item1, '=>', item2)

a => apple
b => banana
c => cherry
d => ***


In [2]:
from itertools import count
counter = count(start=789)
for _ in range(10):
    print(next(counter))

789
790
791
792
793
794
795
796
797
798


In [3]:
from itertools import count

counter = count(1, 0.25)
for _ in range(10):
    print(next(counter))

1
1.25
1.5
1.75
2.0
2.25
2.5
2.75
3.0
3.25


In [4]:
from itertools import cycle
sizes = ['S', 'M', 'L']
sc = cycle(sizes)
#next(sc), next(sc), next(sc), next(sc), next(sc)
for num in range(1, 22):
    print(next(sc), end=' ')

S M L S M L S M L S M L S M L S M L S M L 

In [6]:
from itertools import islice
list(islice(fibonacci(), 50, 60))

[20365011074,
 32951280099,
 53316291173,
 86267571272,
 139583862445,
 225851433717,
 365435296162,
 591286729879,
 956722026041,
 1548008755920]

In [7]:
from itertools import count, islice

for num in islice(count(1, 0.25), 13, 17):
    print(num)

4.25
4.5
4.75
5.0


In [8]:
# some produce a finite sequence from an infinite sequence
from itertools import islice, cycle
colors = cycle(['red', 'white', 'blue']) # infinite
limited = islice(colors, 0, 5)

for color in limited:
    print(color, end= ' ')

red white blue red white 

In [9]:
from itertools import chain
rank = list(range(2, 11))
#picture = { 'J': 'Jack', 'Q': 'Queen', 'K': 'King', 'A': 'Ace' }
picture = list('JQKA')

#for card in chain(rank, picture):
    #print(card, end=' ')
    
list(chain(rank, picture))

[2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']

In [10]:
from itertools import filterfalse
# filter out items for which predicate is False
numbers = [7, 12, 20, 23, 32, 44]
list(filterfalse(lambda x: x % 2, numbers))

[12, 20, 32, 44]

In [11]:
# filters elements, returning only those that have a corresponding
# element that evaluates to True
from itertools import compress
words = ['how', 'now', 'brown', 'cow']
counts = [13, '', 'x', None]
list(compress(words, counts))

['how', 'brown']

In [12]:
# accumulate sums, or other binary functions
from itertools import accumulate
list(accumulate([3, 5, 10, 21]))
#help(accumulate)

[3, 8, 18, 39]

In [13]:
list(accumulate(range(1, 10), lambda x, y: x * y))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

In [14]:
# Amortize a 5% loan of 1000 with 4 annual payments of 250
cashflows = [1000, 250, 250, 250, 250]
list(accumulate(cashflows, lambda bal, pmt: bal * 1.05 - pmt))

[1000, 800.0, 590.0, 369.5, 137.97500000000002]